In [4]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)


StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c4ace404-a7c2-4dc5-99bb-0636e1d85433)

In [5]:
df= df.select("value")

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 7, Finished, Available, Finished)

In [6]:
display(df)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 24353fbb-81ca-4280-9178-da18fc6b1062)

In [7]:
from pyspark.sql.functions import explode
df_exploded =df.select(explode(df["value"]).alias("json_object"))


StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 9, Finished, Available, Finished)

In [8]:
display(df_exploded)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1e9b24d8-c1c4-4b2c-a748-9c338fc24077)

In [9]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 11, Finished, Available, Finished)

In [21]:
print(json_list[1])

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 23, Finished, Available, Finished)

{"json_object":{"datePublished":"2022-11-21T21:55:00.0000000Z","description":"Heavy Rains: IMD Issues Orange Alert For Maharashtra And These States Ganesh Chaturthi 2024: Top Quotes of Gurudev Sri Sri Ravi Shankar Happy Teachers’ Day: 10 Quotes on Master By Gurudev Sri ...","name":"News","provider":[{"_type":"Organization","name":"India.com"}],"url":"https://www.india.com/news/"}}


In [22]:
import json
new_json= json.loads(json_list[1])

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 24, Finished, Available, Finished)

In [23]:
print(new_json)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 25, Finished, Available, Finished)

{'json_object': {'datePublished': '2022-11-21T21:55:00.0000000Z', 'description': 'Heavy Rains: IMD Issues Orange Alert For Maharashtra And These States Ganesh Chaturthi 2024: Top Quotes of Gurudev Sri Sri Ravi Shankar Happy Teachers’ Day: 10 Quotes on Master By Gurudev Sri ...', 'name': 'News', 'provider': [{'_type': 'Organization', 'name': 'India.com'}], 'url': 'https://www.india.com/news/'}}


In [24]:
title=[]
description=[]
category=[]
url=[]
image=[]
provider=[]
datepublished=[]

for json_str in json_list:
    try:
        article=json.loads(json_str)

        if article["json_object"].get("category") and article["json_object"].get("image",{}).get("thumbnail",{}).get("contentUrl"):

         title.append(article["json_object"]["name"])
         description.append(article["json_object"]["description"])
         category.append(article["json_object"]["category"])
         url.append(article["json_object"]["url"])
         image.append(article["json_object"]["image"]["thumbnail"]["contentUrl"])
         provider.append(article["json_object"]["provider"][0]['name'])
         datepublished.append(article["json_object"]["datePublished"])

    except Exception as e:
        print(f"Error processing JSON object : {e}")

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 26, Finished, Available, Finished)

In [25]:
title

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 27, Finished, Available, Finished)

["Man United transfer news LIVE Man Utd injury latest plus Antony update and Adrien Rabiot 'talks'",
 'Chelsea injury update: Cole Palmer, Reece James and Malo Gusto latest news and return dates',
 'Transfer deadline day live updates: Latest news on Arsenal, Liverpool, Manchester United, Chelsea and more',
 'NBA News: Lakers hire new coach to develop Bronny James in G League',
 'Live AFL finals scores 2024, Sydney Swans vs GWS Giants, First Qualifying Final: Updates, stats, blog, start time, teams, latest news',
 'Harris widens lead over Trump after debate, new Reuters/Ipsos poll shows',
 'Kamala Harris edges out Donald Trump by 5 points in latest poll',
 'Car News That Mattered This Week (September 2-6): New Car Launches, Upcoming Cars Teased And More',
 'On 9/11 anniversary, Biden, Harris Trump visit New York site together; Harris & Trump shake hands',
 'Southampton vs Man Utd: Latest team news, predicted lineup and injuries',
 'Delhi News Live Updates: Supreme Court to deliver verdi

In [26]:
from pyspark.sql.types import StructType, StructField, StringType

#combine the lists
data= list(zip(title, description,category, url,image,provider,datepublished))

schema= StructType([
    StructField("title", StringType(), True),
     StructField("description", StringType(), True),
      StructField("category", StringType(), True),
       StructField("url", StringType(), True),
        StructField("image", StringType(), True),
         StructField("provider", StringType(), True),
          StructField("datepublished", StringType(), True),
])

#define schema
df_cleaned=spark.createDataFrame(data,schema=schema)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 28, Finished, Available, Finished)

In [27]:
display(df_cleaned)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5047ad6-9d8c-457e-a738-07bcc0821b99)

In [30]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datepublished"), "dd-MM-yyyy"))


StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 32, Finished, Available, Finished)

In [32]:
display(df_cleaned_final.limit(5))

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e677ad9e-7090-474e-972d-cb3a9729c812)

In [33]:
df_cleaned_final.write.format("delta").saveAsTable("bing_lake_db.tbl_latest_news")

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 35, Finished, Available, Finished)

In [34]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, 08bce4c5-f976-4061-810d-98b0f5bf5d37, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 521e2db0-a270-4aa7-890c-cb9265679e1e)